# San Francisco Voter Guide Debate
This notebook uses the OpenAI assistant API to create a debate between a proponent and opponent on each issue in the 2024 San Francisco election. </br>
Each is seeded with the [official city election guide](https://voterguide.sfelections.org/local-ballot-measures). <br>
The debate instructions are included below for transparency. <br>
To change the measure and the number of rounds, adjust below. 


In [20]:
# a, b, c, d, e, f, g this year
issue = "a"
assert issue in ["a", "b", "c", "d", "e", "f", "g"]
# Debate
num_rounds = 3
debate_instructions="A debate is an organized argument or contest of ideas in which the \
    participants discuss a topic from two opposing sides.  Each side will show in an organized \
    and clever way why they believe to have the right answers.  They will use examples and \
    evidence to support their ideas while working towards a conclusion. \
    The aim of a debate is to convince the opposition that you are right. \
    When the two sides agree on the subject or when one side's arguments are more convincing \
    than the other side that is when the debate comes to a close. \
    Be concise and informative, do not repeat information already stated, and admit when you are wrong and when the other debater is right. \
    Respond to the Moderator's questions and to the other debater's arguments. "


# Setup
Some helper functions from the OpenAI Assistant cookbook: https://cookbook.openai.com/examples/assistants_api_overview_python

In [21]:
import time

def wait_on_run(run, thread, client):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

Download the appropriate issue

In [22]:
import os
import subprocess

os.makedirs("downloads", exist_ok=True)
if not os.path.exists(f"downloads/measure-{issue}.html"):
    subprocess.Popen(["wget", f"https://voterguide.sfelections.org/local-ballot-measures/measure-{issue}", "-O", f"downloads/measure-{issue}.html"])

# Debate
Create assistant, will switch sides to debate itself

In [23]:
from openai import OpenAI

client = OpenAI()

# Upload the file that you want
file = client.files.create(file=open(f"downloads/measure-{issue}.html", "rb"), purpose="assistants")
# Make your debaters
debater = client.beta.assistants.create(
    name="Debater",
    instructions=debate_instructions,
    tools=[{"type": "retrieval"}],
    model="gpt-4-1106-preview",
    file_ids=[file.id]
)


As a user, act as the moderator and prompt the debater

In [29]:
# Opening prompt, one thread for all messages
thread = client.beta.threads.create()

# Let em at it!
for i in range(num_rounds):
    if i == 0: 
        print("User: Start debate")
        message = client.beta.threads.messages.create(
            thread_id=thread.id, 
            role="user", 
            content = f"Moderator: Please concisely summarize Measure {issue} and your stance on it"
        )
    # Continue debate
    elif i < num_rounds - 1:
        print(f"User: Please respond")
        message = client.beta.threads.messages.create(
            thread_id=thread.id, 
            role="user", 
            content = f"Moderator: Please respond to the other debator's arguments and introduce novel arguments to make your case"
        )
    # Closing remarks
    else:
        print(f"User: Closing remarks")
        message = client.beta.threads.messages.create(
            thread_id=thread.id, 
            role="user", 
            content = f"Moderator: Please provide closing remarks on your stance on Measure {issue}"
        )
    # Run assistant as both sides
    for d in range(2):
        print(f"Round {d} for Debater {d}")
        if d % 2 == 0: 
            additional_instructions = f"You support Measure {issue}. You are debating someone who opposes it."
        else:
            additional_instructions = f"You oppose Measure {issue}. You are debating someone who supports it."
        run = client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=debater.id,
            additional_instructions=additional_instructions
        )
        run = wait_on_run(run, thread, client)

User: Start debate
Round 0 for Debater 0
Round 1 for Debater 1
User: Please respond
Round 0 for Debater 0
Round 1 for Debater 1
User: Closing remarks
Round 0 for Debater 0
Round 1 for Debater 1


# Results
Take a look at the results.
Optionally export as an audio file using OpenAI's [TTS model](https://platform.openai.com/docs/guides/text-to-speech)

In [28]:
os.makedirs("outputs", exist_ok=True)

# Gather messages
messages = client.beta.threads.messages.list(thread_id=thread.id)
messages.data.reverse()
moderator = [x.content[0].text.value for x in messages.data[0::3]]
d1 = [x.content[0].text.value for x in messages.data[1::3]]
d2 = [x.content[0].text.value for x in messages.data[2::3]]
# TODO clean up  Preface each of your answers with Debater for Measure {issue}: debater heading and the weird [source] thing
# Print to a file
with open(f"outputs/debate_{issue}.txt", "w") as f:
    for msg in messages.data:
        f.write(msg.content[0].text.value + "\n\n")
        # print(msg.content[0].text.value)

# Turn them into audio
settings = [(moderator, "moderator", "onyx"), (d1, "d1", "nova"), (d2, "d2", "alloy")]
for msgs, filename_out, voice in settings:
  for (i, x) in enumerate(msgs):
    fout = f"outputs/{filename_out}_{i}.mp3"
    moderator_response = client.audio.speech.create(
      model="tts-1",
      voice=voice,
      input=x
    )
    moderator_response.stream_to_file(fout)


/var/folders/1z/9qprdryd55qgw3zx9n01k5jc0000gn/T/ipykernel_9124/4061108048.py:26: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  moderator_response.stream_to_file(fout)


In [ ]:
import